In [ ]:
!pip install -U langchain langchain-core langchain-community langchain-google-genai langchain-openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from google.colab import userdata # 코랩 Secrets 기능 사용을 위한 임포트
from langchain_google_genai import ChatGoogleGenerativeAI
# 코랩 Secrets에서 GOOGLE_API_KEY를 가져오기
try:
    google_api_key = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = google_api_key
    print("GOOGLE_API_KEY가 환경 변수로 설정되었습니다.")
except userdata.SecretNotFoundError:
    print("오류: 'GOOGLE_API_KEY' 비밀을 코랩 Secrets에 설정해주세요. ")
    print("설정 방법: 왼쪽 사이드바의 열쇠 모양 아이콘 클릭 -> 'GOOGLE_API_KEY' 이름으로 키 저장.")
    exit() # API 키가 없으면 스크립트 종료
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")
print(f"Selected LLM: {llm.__class__.__name__}")

GOOGLE_API_KEY가 환경 변수로 설정되었습니다.
Selected LLM: ChatGoogleGenerativeAI


In [ ]:
#데이터베이스 저장된 대화 기록 확인
import sqlite3
import json # 'message' 컬럼의 JSON 문자열을 파싱하기 위함
def get_history_from_db(session_id):
  db_path = f"/content/drive/MyDrive/LangChain/{session_id}_chat.db"
  conn = sqlite3.connect(db_path)
  cursor = conn.cursor()
  print(f"데이터베이스 '{db_path}'에 성공적으로 연결되었습니다.")

  cursor.execute("SELECT * FROM message_store")
  print("\n대화 기록")
  rows = cursor.fetchall()
  for row in rows:
    msg_id, session_id, message_json = row
    try:
        # 'message' 컬럼은 JSON 문자열이므로 파싱
        message_data = json.loads(message_json)
        msg_type = message_data.get("type", "unknown")

        # LangChain 메시지의 실제 내용은 'data' 딕셔너리 안에 'content' 키로 저장
        msg_content = message_data.get("data", {}).get("content", "[내용 없음]")

        print(f"ID: {msg_id}, Session: {session_id}")
        print(f"  Type: {msg_type.upper()}")
        print(f"  Content: {msg_content}")
        print("-" * 40)
    except json.JSONDecodeError:
        print(f"ID: {msg_id}, Session: {session_id}, Message: {message_json[:100]}...") # JSON 파싱 오류 시 원본 출력
        print("-" * 40)

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
store_histories = {} # SQLChatMessageHistory 객체를 저장
# 데이터베이스에서 대화내용 가져오는 함수
def get_chat_history(session_id):
  db_path = f"/content/drive/MyDrive/LangChain/{session_id}_chat.db"
  if session_id not in store_histories:
      print(f"{session_id} 에 대한 SQLChatMessageHistory 생성 및 연결")
      sql_chat_history = SQLChatMessageHistory(
            session_id=session_id,
            connection=f"sqlite:///{db_path}"
      )
      store_histories[session_id] = sql_chat_history
  return store_histories[session_id] # SQLChatMessageHistory 객체 반환

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 한국어 챗봇입니다."),
    MessagesPlaceholder(variable_name="history"), # 현재 대화 기록
    ("human", "{input}"),
])

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

# ConversationSummaryBufferMemory 사용
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=200,  # 요약의 기준이 되는 토큰 길이 설정
    return_messages=True,
)

<ipython-input-7-85111161>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


# ConversationChain 사용

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain
conversation = ConversationChain(
    llm = llm,
    memory=memory,
    prompt=prompt,
)

<ipython-input-8-3524113137>:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


In [ ]:
def chatbot(session_id):
  print("\n--- 대화형 챗봇 시작 ---")
  print("종료하려면 'exit' 또는 'quit'을 입력하세요.")
  print("챗봇: 안녕하세요. 무엇을 도와드릴까요?")
  sql_chat_history = get_chat_history(session_id)
  while True:
    user_input = input("입력: ")
    if user_input.lower() in ["exit", "quit"]:
        print("챗봇을 종료합니다.")
        break
    print("챗봇 (답변 생성 중)...")
    try:
      response = conversation.invoke({"input":user_input})['response']
      print(response)

      # 사용자 메시지 추가
      sql_chat_history.add_user_message(
        user_input
      )
      # AI 메시지 추가
      sql_chat_history.add_ai_message(response)
      print("\n--- Summarizer Memory의 요약이 업데이트되었습니다. ---")
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

In [ ]:
chatbot('session1')


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
session1 에 대한 SQLChatMessageHistory 생성 및 연결
입력: 서울에 대해 간략하게 설명해줘
챗봇 (답변 생성 중)...
서울은 대한민국의 수도이자 가장 큰 도시입니다.  약 1000만 명이 넘는 인구가 살고 있으며, 역사와 현대가 공존하는 역동적인 도시입니다.  고궁과 같은 유구한 역사 유적과 최첨단 기술, 번화한 상업 지역, 그리고 다양한 문화 시설들이 조화롭게 어우러져 있습니다.  세계적인 수준의 음식, 쇼핑, 엔터테인먼트를 즐길 수 있으며,  한국의 정치, 경제, 문화의 중심지 역할을 하고 있습니다.  짧게 말하면, 서울은 역사와 현대가 만나는 매력적인 도시입니다!

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: 내 이름은 테디야
챗봇 (답변 생성 중)...
반갑습니다, 테디님!  무엇을 도와드릴까요? 😊

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: quit
챗봇을 종료합니다.


In [ ]:
#이전 대화를 기억함
chatbot('session2')


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
session2 에 대한 SQLChatMessageHistory 생성 및 연결
입력: 내 이름을 기억해?
챗봇 (답변 생성 중)...
네, 테디님!  이름 기억하고 있습니다.  다시 뵙게 되어 반갑습니다!  무슨 말씀이든 편하게 해주세요.

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: 이번엔 버스에 대해 알려줄래?
챗봇 (답변 생성 중)...
네, 버스에 대해 알려드릴게요!  어떤 정보를 원하시는지 좀 더 자세히 말씀해주시면 더욱 정확하게 답변해 드릴 수 있습니다.  예를 들어,  서울 시내 버스에 대해 궁금하신지,  어떤 종류의 버스 정보가 필요하신지 (노선, 요금, 이용 방법 등) 알려주시면 좋겠습니다.

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: 버스의 종류에 대해 알려줘
챗봇 (답변 생성 중)...
버스의 종류는 여러 기준으로 나눌 수 있습니다.  크게는 운행 지역, 운행 방식, 크기 등으로 나눌 수 있죠.  한국, 특히 서울을 기준으로 설명해 드리겠습니다.

**1. 운행 지역에 따른 분류:**

* **시내버스:** 서울 시내를 운행하는 버스로, 일반적으로 번호가 매겨져 있습니다.  간선버스, 지선버스, 마을버스로 세분화됩니다.
    * **간선버스:** 주요 간선도로를 운행하며, 장거리 이동에 적합합니다.  번호는 파란색입니다.
    * **지선버스:** 간선버스를 보조하여, 골목길 등을 운행합니다.  번호는 초록색입니다.
    * **마을버스:** 특정 지역을 중심으로 운행하는 소형 버스입니다.  번호는 빨간색이나 노란색입니다.

* **광역버스:** 서울과 인근 지역을 연결하는 버스입니다.  서울에서 경기도, 인천 등으로 이동할 때 이용합니다.

* **시외버스:** 서울과 다른 시·도를 연결하는 장거리 버스입니다.  고속버스터미널 등

In [ ]:
#메모리 확인_이전 대화들이 모두 저장되어 있음
conversation.memory.load_memory_variables({})['history']

[SystemMessage(content="The human asks the AI for a brief explanation of Seoul. The AI responds that Seoul is the capital and largest city of South Korea, with a population exceeding 10 million. It's described as a dynamic city where history and modernity coexist, blending historical sites like palaces with cutting-edge technology, bustling commercial areas, and diverse cultural facilities.  The AI highlights Seoul's world-class food, shopping, and entertainment, and its role as the center of South Korean politics, economics, and culture. In short, the AI characterizes Seoul as a captivating city where history and modernity meet.  The conversation then shifts to the human introducing themselves as Teddy, and the AI acknowledging and remembering the name.  Finally, the human asks about different types of buses, and the AI provides a detailed explanation categorizing buses by operating area (city, metropolitan, intercity, airport), operation method (regular, express, direct seated), and 

In [ ]:
#데이터베이스 확인
get_history_from_db("session2")

데이터베이스 '/content/drive/MyDrive/LangChain/session2_chat.db'에 성공적으로 연결되었습니다.

대화 기록
ID: 1, Session: session2
  Type: HUMAN
  Content: 내 이름을 기억해?
----------------------------------------
ID: 2, Session: session2
  Type: AI
  Content: 네, 테디님!  이름 기억하고 있습니다.  다시 뵙게 되어 반갑습니다!  무슨 말씀이든 편하게 해주세요.
----------------------------------------
ID: 3, Session: session2
  Type: HUMAN
  Content: 이번엔 버스에 대해 알려줄래?
----------------------------------------
ID: 4, Session: session2
  Type: AI
  Content: 네, 버스에 대해 알려드릴게요!  어떤 정보를 원하시는지 좀 더 자세히 말씀해주시면 더욱 정확하게 답변해 드릴 수 있습니다.  예를 들어,  서울 시내 버스에 대해 궁금하신지,  어떤 종류의 버스 정보가 필요하신지 (노선, 요금, 이용 방법 등) 알려주시면 좋겠습니다.
----------------------------------------
ID: 5, Session: session2
  Type: HUMAN
  Content: 버스의 종류에 대해 알려줘
----------------------------------------
ID: 6, Session: session2
  Type: AI
  Content: 버스의 종류는 여러 기준으로 나눌 수 있습니다.  크게는 운행 지역, 운행 방식, 크기 등으로 나눌 수 있죠.  한국, 특히 서울을 기준으로 설명해 드리겠습니다.

**1. 운행 지역에 따른 분류:**

* **시내버스:** 서울 시내를 운행하는 버스로, 일반적으로 번호가 매겨져 있습니다.  간선버스,

#메모리 직접 사용

In [ ]:
def chatbot(session_id):
  print("\n--- 대화형 챗봇 시작 ---")
  print("종료하려면 'exit' 또는 'quit'을 입력하세요.")
  print("챗봇: 안녕하세요. 무엇을 도와드릴까요?")
  sql_chat_history = get_chat_history(session_id)
  while True:
    user_input = input("입력: ")
    if user_input.lower() in ["exit", "quit"]:
        print("챗봇을 종료합니다.")
        break
    # config 설정. 세션 ID 기준으로 대화 기록
    config = {"configurable": {"session_id": session_id}}
    print("챗봇 (답변 생성 중)...")
    try:
      # ConversationSummaryBufferMemory에서 요약된 내용 로드
      history = memory.load_memory_variables({})['history']
      print("\n--- history에 저장된 내용 ---")
      print(history)
      print("---history 내용 출력 끝---\n")
      config = {"configurable": {"session_id": session_id}}
      inputs = {"input": user_input, "history":history}
      chain = prompt|llm
      response = chain.invoke(inputs, config).content
      print(response)
      # 사용자 입력과 AI의 답변을 ConversationSummaryMemory에 전달하여 요약 업데이트
      memory.save_context(
                {"input": user_input}, # 사용자 입력
                {"output": response} # AI의 답변
      )

      # 사용자 메시지 추가
      sql_chat_history.add_user_message(
        user_input
      )
      # AI 메시지 추가
      sql_chat_history.add_ai_message(response)
      print("\n--- Summarizer Memory의 요약이 업데이트되었습니다. ---")
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

In [ ]:
chatbot('session3')


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
session3 에 대한 SQLChatMessageHistory 생성 및 연결
입력: 그럼 이번엔 지하철에 대해서 간략히 설명해줘
챗봇 (답변 생성 중)...

--- history에 저장된 내용 ---
[SystemMessage(content="The human asks the AI for a brief explanation of Seoul. The AI responds that Seoul is the capital and largest city of South Korea, with a population exceeding 10 million. It's described as a dynamic city where history and modernity coexist, blending historical sites like palaces with cutting-edge technology, bustling commercial areas, and diverse cultural facilities.  The AI highlights Seoul's world-class food, shopping, and entertainment, and its role as the center of South Korean politics, economics, and culture. In short, the AI characterizes Seoul as a captivating city where history and modernity meet.  The conversation then shifts to the human introducing themselves as Teddy, and the AI acknowledging and remembering the name.  Finally, the human asks about different types o

In [ ]:
#메모리 확인
memory.load_memory_variables({})['history']

[SystemMessage(content="The human, Teddy, asks the AI for a brief explanation of Seoul, receiving a detailed response covering its status as South Korea's capital and largest city, its population, dynamic blend of history and modernity, and its role as a center of politics, economics, and culture.  Teddy then inquires about different types of buses in Seoul, receiving a categorized explanation. After thanking the AI in Korean, Teddy asks about the Seoul subway system, getting a description of its extensive network, efficiency, and features like Wi-Fi, while acknowledging peak-hour crowding and limited operating hours. Finally, Teddy asks for a brief explanation of Jeju Island, and the AI responds by describing it as a volcanic island in the south of South Korea, renowned for its beautiful natural scenery, unique culture, UNESCO World Heritage status, and diverse attractions, including Hallasan Mountain, oreum (parasite volcanoes), gotjawal forests, and stunning coastal areas. The AI hi

In [ ]:
#데이터베이스 확인
get_history_from_db('session3')

데이터베이스 '/content/drive/MyDrive/LangChain/session3_chat.db'에 성공적으로 연결되었습니다.

대화 기록
ID: 1, Session: session3
  Type: HUMAN
  Content: 그럼 이번엔 지하철에 대해서 간략히 설명해줘
----------------------------------------
ID: 2, Session: session3
  Type: AI
  Content: 서울 지하철은 서울과 수도권 지역을 연결하는 광범위한 대중교통 시스템입니다.  1호선부터 9호선까지, 그리고 수도권 전철 여러 노선을 포함하여 매우 효율적이고 편리하게 도시 곳곳을 이동할 수 있도록 해줍니다.  대부분의 노선은 24시간 운영되지는 않지만, 새벽까지 운행하며, 환승 시스템이 잘 구축되어 있어 다른 노선으로의 이동이 편리합니다.  지하철역들은 대부분 지상과 연결되어 있으며, 주요 상업지역과 관광지와 가까이 위치해 있습니다.  최근에는 무선 인터넷(Wi-Fi) 설비도 잘 갖춰져 있어 이동 중에도 편리하게 인터넷을 이용할 수 있습니다.  하지만 혼잡 시간대에는 매우 붐빌 수 있다는 점을 유의하시는 것이 좋습니다.
----------------------------------------
ID: 3, Session: session3
  Type: HUMAN
  Content: 제주도에 대해서도 간략히 설명해줄래?
----------------------------------------
ID: 4, Session: session3
  Type: AI
  Content: 제주도는 대한민국 남쪽에 위치한 화산섬으로, 아름다운 자연경관과 독특한 문화로 유명합니다.  한라산이라는 휴화산을 중심으로 섬 전체가 유네스코 세계자연유산으로 등재되어 있으며,  오름이라 불리는 기생화산들과 곶자왈이라는 특이한 숲, 그리고 맑고 푸른 바다가 어우러져 아름다운 풍경을 자랑합니다.  

제주도는 다양한 관광 명소를 갖추고 있는데, 성산일출봉, 만장굴

In [ ]:
chatbot('session4')


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
session4 에 대한 SQLChatMessageHistory 생성 및 연결
입력: 우리가 어떤 대화를 나눴는지 기억해?
챗봇 (답변 생성 중)...

--- history에 저장된 내용 ---
[SystemMessage(content="The human, Teddy, asks the AI for a brief explanation of Seoul, receiving a detailed response covering its status as South Korea's capital and largest city, its population, dynamic blend of history and modernity, and its role as a center of politics, economics, and culture.  Teddy then inquires about different types of buses in Seoul, receiving a categorized explanation. After thanking the AI in Korean, Teddy asks about the Seoul subway system, getting a description of its extensive network, efficiency, and features like Wi-Fi, while acknowledging peak-hour crowding and limited operating hours. Finally, Teddy asks for a brief explanation of Jeju Island, and the AI responds by describing it as a volcanic island in the south of South Korea, renowned for its beautiful natural scenery, 